#     Final Project - Stock Scraper

# Description : 
The main aim of the application is to create a web scraper that scrapes a stock website and gets the user the most active, the gainers and the losers in the stock market. The program scrapes the webpages to fetch the details of the stocks and stores the values in a csv file. The values include Open, Previos close, Market Cap and volume. These values are stored in the csv along with the status of the stock and its ticker symbol. 

The program at first, takes a few seconds to enter the data into the csv file. Then it asks the user to present an input symbol and the corresponding values are printed on the console. The user is asked about continuing the process again. If given N or n, then the program stops.

website used to fetch the stock specific values : https://finance.yahoo.com/quote/

# Procedure : 
- Import the necessessary libraries
- Run the program
- The program automatically scrapes the stock details
- These values are then stored into a csv file named stocks.csv
- The program asks the user for the input symbol.
- It fetches the values from the csv file and displays it on the console
- The user is asked about performing the process again, Y or y askes the user to enter another symbol
- Typing n or N ends the process and exits the program

In [ ]:
import urllib.request, urllib.parse, urllib.error #Importing these to retrieve the webpage html content
from bs4 import BeautifulSoup # importing to scrape the data from the webpage
import ssl # importing to check the certificate of the website
import csv # importing to report the output to a comma seperated file

i = []
j = 0
super_dict = {}
test_dict = {}

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE # verifying the certificate
html = urllib.request.urlopen("https://money.cnn.com/data/hotstocks/", context=ctx).read() # open the webpage and get the html content of the webpage requested as a request object
soup = BeautifulSoup(html, 'lxml') # parsing the object using beautiful soup to traverse through the html document
csv_file = open('stocks.csv','w') # creates a csv file with the write mode to add lines to the file
csv_writer = csv.writer(csv_file,lineterminator='\n') # used to remove the newline in the csv file
csv_writer.writerow(['Status','Ticker Symbol','Stock Name','Open','PrevClose','Volume','Market Cap']) # defines the first row/ header fields in the csv file
list_of_h3 = soup.find('div',id="wsod_hotStocks") # use beautiful soup to scrape the first div tag with the respective id
h3 = list_of_h3.find_all('h3') # retrieves all the h3 tags in the scraped div tag
for h4 in h3: # looping through the list generated
    i.append(h4.text) # storing the header values in another list

first_div = soup.find_all('table',class_="wsod_dataTable wsod_dataTableBigAlt") # use beautiful soup to find all the tables with the respective class

print("Which stock are you interested in : ")
print()
for table_list in first_div: #looping through the list of table generated above
    stock_status = i[j] # Assign the status to a variable
    print(stock_status)
    table_rows = table_list.find_all('tr') # obtain a list of all the tr tags in that specific table
    for tr in table_rows:
        td = tr.find_all('td') # obtain all the td in that specific tr tag
        row = [i.text for i in td] # store the value of the text in the td tag into a variable
        if len(row) > 0: #Check if the length of the row is greater than 0
            print(row[0]) 
            sample = row[0].split(" ",1) #split the row int0 2 parts (symbol and stock name) using " "
            super_dict[sample[0]] = sample[1] # form a dictionary of symbol - stock name pair
            if j >= 0:
                test_dict[sample[0]] = stock_status # storing the status of the stock inside a dictionary
            sample = [] # clear off the list to store the values of the next td
    print()
    j += 1 

for j in test_dict.keys(): # looping through the ticker symbols
    q = [j,j] # store in list the same values to use for link redirection
    link_merge = "?p=".join(q) # join the list using the term given to achieve a link format structure
    stock_link = f"https://finance.yahoo.com/quote/{link_merge}&.tsrc=fin-srch-v1" # store the link another variable

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    html = urllib.request.urlopen(stock_link, context=ctx).read() # fetch the content of the website to a html format
    yahoo_soup = BeautifulSoup(html, 'lxml') # parse the page

    value_table = yahoo_soup.find('table',class_="W(100%)") # find the first table with the respective class

    value_rows = value_table.find_all('tr') # finds all the tr tags in that table
    for sample_td in value_rows: 
        value_td = sample_td.find_all('td') # finds all the td tags in the tr tag being iterated
        for val in value_td:    
            prev_close = val.get('data-test',None) # get the value of the specified attribute
            open_value = val.get('data-test',None) 
            volume_value = val.get('data-test',None) 
            if open_value == "OPEN-value":
                stock_open_value = val.text # store the open value from the table
            elif prev_close == "PREV_CLOSE-value":
                stock_prev_close_value = val.text # Store the previous close value from the table
            elif volume_value == "TD_VOLUME-value":
                stock_volume_value = val.text # store the volume value from the table
            else:
                continue
    value_table_1 = yahoo_soup.find('table',class_="W(100%) M(0) Bdcl(c)") # find the first table with the specified attribute
    value_row_1 = value_table_1.find('tr')   # find the first tr tag  
    value_td_1 = value_row_1.find_all('td') # find the first td tag
    for temp_val in value_td_1:
        market_cap = temp_val.get('data-test',None) # get the value of the specified attribute
        if market_cap == "MARKET_CAP-value":
            stock_market_value = temp_val.text # print the value of the market cap value
        else:
            continue # continue, if the value cannot be found
    csv_writer.writerow([test_dict[j],j,super_dict[j],stock_open_value,stock_prev_close_value,stock_volume_value,stock_market_value]) # write the value into the csv file

csv_file.close() # close the csv file handle after completing the process

store_temp = {}
super_temp = {}
with open('stocks.csv','r') as csv_file: # open the same csv file in read mode
    csv_reader = csv.reader(csv_file)

    next(csv_reader) # skip the first row ("the headings")

    for line in csv_reader: # looping through the values 
        store_temp["stock_name"] = line[2] # storing the values in a dictionary with specific key values
        store_temp["OPEN"] = line[3]
        store_temp["PREV CLOSE"] = line[4]
        store_temp["VOLUME"] = line[5]
        store_temp["MARKET CAP"] = line[6]
        super_temp[line[1]] = store_temp # storing the above generated dictionary as a value to the key symbol in a dictionary
        store_temp = {} # clear the temporary dictionary for the next value
    while True: # to check if the user wants to continue or not
        while True: # to check if the user has entered a proper symbol or not
            user_inp = input("User inputs : ")
            user_inp = user_inp.upper() # convert to uppercase to check

            if user_inp in super_temp.keys(): # checking in the lsit of keys in dictionary
                stock_details = super_temp[user_inp] # storing the value in another value
                print("The data for",user_inp,stock_details["stock_name"],"is the following : ")
                print() 
                print("OPEN : ",stock_details["OPEN"]) # printing out the details from the dictionary
                print("PREV CLOSE : ",stock_details["PREV CLOSE"])
                print("VOLUME : ",stock_details["VOLUME"])
                print("MARKET CAP : ",stock_details["MARKET CAP"])
                print()
            else:
                print("Ticker symbol not found in the csv. Please enter a proper value") # if the user has not entered a proper symbol
                print()
                continue
            break

        user_temp = input("Do you want to view another stock value (Y/N) : ")
        print()
        if user_temp.upper() == "Y": # looping if the user wants to continue the process
            continue
        else:
            break
csv_file.close() # close the csv file handle after completing the process